In [1]:
import struct
import numpy as np
import tensorflow as tf
import skimage
from PIL import Image
from skimage.transform import resize
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def read_record_ETL8G(f):
    s = f.read(8199)
    r = struct.unpack('>2H8sI4B4H2B30x8128s11x', s)
    iF = Image.frombytes('F', (128, 127), r[14], 'bit', 4)
    iL = iF.convert('L')
    return r + (iL,)

In [6]:
if __name__ == '__main__':
    data = []
    jis = []
    for i in range(1, 33):
        with open('./ETL8G/ETL8G_{:02d}'.format(i), 'rb') as f:
            for j in range(5*956):
                r = read_record_ETL8G(f)
                if 0x2421 <= r[1] <= 0x2473:
                    data.append((
                        resize(
                            np.asarray(r[-1])/9*255,
                            (32, 32),
                            mode='constant')/255).flatten())
                    jis.append(r[1])


In [26]:
jis = pd.DataFrame({'jis': jis})
classmapping = pd.read_csv('./classmapping.csv', dtype={'class': np.int32}, encoding='cp932')
labels = pd.merge(
            jis,
            classmapping,
            how='left',
            on='jis')['class'].values

In [29]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.39,
    zoom_range=0.2,
    data_format='channels_first')

In [30]:
data = np.array(data, dtype=np.float32)
reshaped_data = data.reshape((12000, 1, 32, 32))
datagen.fit(reshaped_data)
generator = datagen.flow(reshaped_data, labels, batch_size=12000)

In [31]:
for i in range(49):
    new_dataset = generator.next()
　   new_data = new_dataset[0].reshape((12000, 1024))
　   new_labels = new_dataset[1]
    data = np.concatenate((data, new_data))
    labels = np.append(labels, new_labels)

In [34]:
np.savez_compressed(
    './hira.npz',
    data=data,
    labels=labels)